# 強化學習基礎 (Reinforcement Learning)

## 學習目標
- 理解強化學習的基本概念
- 實作 Policy Gradient
- 學習 Actor-Critic 方法
- 解決 CartPole 問題

## 對應課程
- [李宏毅 ML 2021 - Reinforcement Learning](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW12: Reinforcement Learning](https://github.com/ga642381/ML2021-Spring/tree/main/HW12)

## 強化學習概述

```
強化學習框架
    Agent ←→ Environment
      │         │
      │ action  │ state, reward
      └─────────┴─────────────

核心概念:
├── State (s): 環境狀態
├── Action (a): 代理動作
├── Reward (r): 獎勵信號
├── Policy (π): 策略函數 π(a|s)
└── Value (V): 價值函數 V(s)

主要方法:
├── Value-based: Q-Learning, DQN
├── Policy-based: REINFORCE, PPO
└── Actor-Critic: A2C, A3C, SAC
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import warnings
warnings.filterwarnings('ignore')

# 嘗試導入 gym
try:
    import gymnasium as gym
    HAS_GYM = True
    print("Gymnasium 已載入")
except ImportError:
    try:
        import gym
        HAS_GYM = True
        print("Gym 已載入")
    except ImportError:
        HAS_GYM = False
        print("請安裝: pip install gymnasium 或 pip install gym")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
torch.manual_seed(42)
np.random.seed(42)

---
## Part 1: 環境介紹 - CartPole

CartPole 是一個經典的控制問題：控制一個小車，使其上的杆子保持平衡。

- **State**: [車位置, 車速度, 杆角度, 杆角速度]
- **Action**: 0=向左, 1=向右
- **Reward**: 每步 +1（杆子沒倒）

In [ ]:
# ========== 簡易 CartPole 環境（不需要 gym）==========

class SimpleCartPole:
    """
    簡化版 CartPole 環境
    
    用於在沒有 gym 時演示 RL 概念
    """
    
    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = self.masspole + self.masscart
        self.length = 0.5
        self.polemass_length = self.masspole * self.length
        self.force_mag = 10.0
        self.tau = 0.02
        
        # 閾值
        self.theta_threshold = 12 * 2 * np.pi / 360
        self.x_threshold = 2.4
        
        self.state = None
        self.steps = 0
    
    def reset(self):
        self.state = np.random.uniform(-0.05, 0.05, size=(4,))
        self.steps = 0
        return self.state.copy()
    
    def step(self, action):
        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag if action == 1 else -self.force_mag
        
        costheta = np.cos(theta)
        sintheta = np.sin(theta)
        
        temp = (force + self.polemass_length * theta_dot**2 * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0/3.0 - self.masspole * costheta**2 / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass
        
        x = x + self.tau * x_dot
        x_dot = x_dot + self.tau * xacc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * thetaacc
        
        self.state = np.array([x, x_dot, theta, theta_dot])
        self.steps += 1
        
        done = (
            x < -self.x_threshold or
            x > self.x_threshold or
            theta < -self.theta_threshold or
            theta > self.theta_threshold or
            self.steps >= 500
        )
        
        reward = 1.0 if not done else 0.0
        
        return self.state.copy(), reward, done, {}
    
    @property
    def observation_space(self):
        class Space:
            shape = (4,)
        return Space()
    
    @property
    def action_space(self):
        class Space:
            n = 2
        return Space()

# 建立環境
if HAS_GYM:
    env = gym.make('CartPole-v1')
else:
    env = SimpleCartPole()

print(f"觀察空間: {env.observation_space.shape}")
print(f"動作空間: {env.action_space.n}")

# 測試環境
state = env.reset() if not HAS_GYM else env.reset()[0]
print(f"\n初始狀態: {state}")

In [ ]:
# ========== 隨機策略測試 ==========

def run_random_policy(env, n_episodes=10):
    """
    用隨機策略測試環境
    """
    rewards = []
    
    for _ in range(n_episodes):
        state = env.reset() if not HAS_GYM else env.reset()[0]
        episode_reward = 0
        done = False
        
        while not done:
            action = np.random.randint(0, env.action_space.n)
            result = env.step(action)
            if HAS_GYM:
                state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                state, reward, done, _ = result
            episode_reward += reward
        
        rewards.append(episode_reward)
    
    return rewards

random_rewards = run_random_policy(env, n_episodes=100)
print(f"隨機策略平均獎勵: {np.mean(random_rewards):.2f} ± {np.std(random_rewards):.2f}")

---
## Part 2: Policy Gradient (REINFORCE)

### 核心公式

目標：最大化期望回報
$$J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta}[R(\tau)]$$

梯度：
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau}\left[\sum_{t=0}^{T} \nabla_\theta \log \pi_\theta(a_t|s_t) \cdot G_t\right]$$

其中 $G_t = \sum_{k=t}^{T} \gamma^{k-t} r_k$ 是從時刻 t 開始的折扣回報。

In [ ]:
# ========== Policy Network ==========

class PolicyNetwork(nn.Module):
    """
    策略網路
    
    輸入: 狀態
    輸出: 動作的概率分布
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super().__init__()
        
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, action_dim)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=-1)
    
    def get_action(self, state, deterministic=False):
        """
        根據狀態選擇動作
        """
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        probs = self.forward(state)
        
        if deterministic:
            action = probs.argmax(dim=-1).item()
        else:
            dist = Categorical(probs)
            action = dist.sample().item()
        
        return action, probs

# 測試
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

policy = PolicyNetwork(state_dim, action_dim).to(device)
print(f"策略網路:")
print(f"  輸入維度: {state_dim}")
print(f"  輸出維度: {action_dim}")
print(f"  參數量: {sum(p.numel() for p in policy.parameters())}")

In [ ]:
# ========== REINFORCE 演算法 ==========

class REINFORCE:
    """
    REINFORCE 演算法（帶 baseline）
    """
    
    def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99):
        self.policy = PolicyNetwork(state_dim, action_dim).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma
        
        # 儲存軌跡
        self.log_probs = []
        self.rewards = []
    
    def select_action(self, state):
        """選擇動作並記錄 log probability"""
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        probs = self.policy(state)
        
        dist = Categorical(probs)
        action = dist.sample()
        
        self.log_probs.append(dist.log_prob(action))
        
        return action.item()
    
    def store_reward(self, reward):
        """儲存獎勵"""
        self.rewards.append(reward)
    
    def update(self):
        """更新策略"""
        # 計算折扣回報
        returns = []
        G = 0
        for r in reversed(self.rewards):
            G = r + self.gamma * G
            returns.insert(0, G)
        
        returns = torch.tensor(returns).to(device)
        
        # 正規化（baseline）
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        
        # 計算損失
        policy_loss = []
        for log_prob, G in zip(self.log_probs, returns):
            policy_loss.append(-log_prob * G)
        
        policy_loss = torch.stack(policy_loss).sum()
        
        # 更新
        self.optimizer.zero_grad()
        policy_loss.backward()
        self.optimizer.step()
        
        # 清空軌跡
        self.log_probs = []
        self.rewards = []
        
        return policy_loss.item()

# 建立 agent
agent = REINFORCE(state_dim, action_dim, lr=1e-3, gamma=0.99)
print("REINFORCE Agent 建立完成")

In [ ]:
# ========== 訓練 REINFORCE ==========

def train_reinforce(env, agent, n_episodes=500, print_every=50):
    """
    訓練 REINFORCE agent
    """
    episode_rewards = []
    avg_rewards = []
    
    for episode in range(n_episodes):
        state = env.reset() if not HAS_GYM else env.reset()[0]
        episode_reward = 0
        done = False
        
        while not done:
            # 選擇動作
            action = agent.select_action(state)
            
            # 執行動作
            result = env.step(action)
            if HAS_GYM:
                next_state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                next_state, reward, done, _ = result
            
            # 儲存獎勵
            agent.store_reward(reward)
            
            state = next_state
            episode_reward += reward
        
        # Episode 結束，更新策略
        agent.update()
        
        episode_rewards.append(episode_reward)
        avg_reward = np.mean(episode_rewards[-100:])
        avg_rewards.append(avg_reward)
        
        if (episode + 1) % print_every == 0:
            print(f"Episode {episode+1}: Reward = {episode_reward:.0f}, Avg(100) = {avg_reward:.1f}")
        
        # 提前結束
        if avg_reward >= 475:
            print(f"\n環境已解決！在 {episode+1} 個 episode 後")
            break
    
    return episode_rewards, avg_rewards

# 訓練
rewards, avg_rewards = train_reinforce(env, agent, n_episodes=500, print_every=50)

In [ ]:
# ========== 可視化訓練結果 ==========

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 每個 episode 的獎勵
axes[0].plot(rewards, alpha=0.3)
axes[0].plot(avg_rewards, linewidth=2)
axes[0].axhline(y=475, color='r', linestyle='--', label='Solved')
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Reward')
axes[0].set_title('Training Progress')
axes[0].legend(['Episode Reward', 'Avg (100)', 'Solved Threshold'])
axes[0].grid(True, alpha=0.3)

# 獎勵分布
axes[1].hist(rewards, bins=30, edgecolor='black')
axes[1].axvline(x=np.mean(rewards), color='r', linestyle='--', label=f'Mean: {np.mean(rewards):.1f}')
axes[1].set_xlabel('Reward')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Reward Distribution')
axes[1].legend()

plt.tight_layout()
plt.show()

---
## Part 3: Actor-Critic

Actor-Critic 結合了 Policy Gradient 和 Value Function：
- **Actor**: 學習策略 $\pi(a|s)$
- **Critic**: 學習價值函數 $V(s)$，用於減少方差

In [ ]:
# ========== Actor-Critic 網路 ==========

class ActorCritic(nn.Module):
    """
    Actor-Critic 網路
    
    共享底層特徵提取，分別輸出：
    - Actor: 動作概率
    - Critic: 狀態價值
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super().__init__()
        
        # 共享層
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        
        # Actor 頭
        self.actor = nn.Linear(hidden_dim, action_dim)
        
        # Critic 頭
        self.critic = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        features = self.shared(x)
        
        action_probs = F.softmax(self.actor(features), dim=-1)
        state_value = self.critic(features)
        
        return action_probs, state_value

# 測試
ac_net = ActorCritic(state_dim, action_dim).to(device)
state_tensor = torch.randn(1, state_dim).to(device)
probs, value = ac_net(state_tensor)

print(f"Actor-Critic 網路:")
print(f"  動作概率: {probs.shape}")
print(f"  狀態價值: {value.shape}")

In [ ]:
# ========== A2C (Advantage Actor-Critic) ==========

class A2C:
    """
    Advantage Actor-Critic
    
    使用 Advantage = R + γV(s') - V(s) 代替回報
    """
    
    def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99, value_coef=0.5, entropy_coef=0.01):
        self.network = ActorCritic(state_dim, action_dim).to(device)
        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)
        
        self.gamma = gamma
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef
        
        # 軌跡儲存
        self.log_probs = []
        self.values = []
        self.rewards = []
        self.entropies = []
    
    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        probs, value = self.network(state)
        
        dist = Categorical(probs)
        action = dist.sample()
        
        self.log_probs.append(dist.log_prob(action))
        self.values.append(value)
        self.entropies.append(dist.entropy())
        
        return action.item()
    
    def store_reward(self, reward):
        self.rewards.append(reward)
    
    def update(self, next_state, done):
        # 計算 bootstrap value
        if done:
            R = 0
        else:
            next_state = torch.FloatTensor(next_state).unsqueeze(0).to(device)
            _, next_value = self.network(next_state)
            R = next_value.item()
        
        # 計算回報
        returns = []
        for r in reversed(self.rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        
        returns = torch.tensor(returns).to(device)
        values = torch.cat(self.values).squeeze()
        log_probs = torch.stack(self.log_probs)
        entropies = torch.stack(self.entropies)
        
        # 計算 advantage
        advantages = returns - values.detach()
        
        # 計算損失
        actor_loss = -(log_probs * advantages).mean()
        critic_loss = F.mse_loss(values, returns)
        entropy_loss = -entropies.mean()
        
        total_loss = actor_loss + self.value_coef * critic_loss + self.entropy_coef * entropy_loss
        
        # 更新
        self.optimizer.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.network.parameters(), 0.5)
        self.optimizer.step()
        
        # 清空
        self.log_probs = []
        self.values = []
        self.rewards = []
        self.entropies = []
        
        return total_loss.item()

# 建立 A2C agent
a2c_agent = A2C(state_dim, action_dim, lr=1e-3)
print("A2C Agent 建立完成")

In [ ]:
# ========== 訓練 A2C ==========

def train_a2c(env, agent, n_episodes=500, print_every=50):
    """
    訓練 A2C agent
    """
    episode_rewards = []
    avg_rewards = []
    
    for episode in range(n_episodes):
        state = env.reset() if not HAS_GYM else env.reset()[0]
        episode_reward = 0
        done = False
        
        while not done:
            action = agent.select_action(state)
            
            result = env.step(action)
            if HAS_GYM:
                next_state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                next_state, reward, done, _ = result
            
            agent.store_reward(reward)
            
            state = next_state
            episode_reward += reward
        
        # 更新
        agent.update(state, done)
        
        episode_rewards.append(episode_reward)
        avg_reward = np.mean(episode_rewards[-100:])
        avg_rewards.append(avg_reward)
        
        if (episode + 1) % print_every == 0:
            print(f"Episode {episode+1}: Reward = {episode_reward:.0f}, Avg(100) = {avg_reward:.1f}")
        
        if avg_reward >= 475:
            print(f"\n環境已解決！在 {episode+1} 個 episode 後")
            break
    
    return episode_rewards, avg_rewards

# 訓練 A2C
a2c_rewards, a2c_avg_rewards = train_a2c(env, a2c_agent, n_episodes=500, print_every=50)

---
## Part 4: 比較不同方法

In [ ]:
# ========== 比較 REINFORCE vs A2C ==========

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(avg_rewards, label='REINFORCE', alpha=0.8)
ax.plot(a2c_avg_rewards, label='A2C', alpha=0.8)
ax.axhline(y=475, color='r', linestyle='--', label='Solved')

ax.set_xlabel('Episode')
ax.set_ylabel('Average Reward (100 episodes)')
ax.set_title('REINFORCE vs A2C on CartPole')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n最終表現:")
print(f"  REINFORCE: {avg_rewards[-1]:.1f}")
print(f"  A2C: {a2c_avg_rewards[-1]:.1f}")

---
## 🎯 總結

### RL 方法比較

| 方法 | 優點 | 缺點 |
|------|------|------|
| **REINFORCE** | 簡單、直觀 | 高方差、樣本效率低 |
| **A2C** | 較低方差 | 可能收斂到局部最優 |
| **PPO** | 穩定、通用 | 超參數較多 |
| **DQN** | 樣本效率高 | 只適用離散動作 |

### 李宏毅 HW12 技巧

```
Policy Gradient 要點:
1. 使用 baseline 減少方差
2. 正規化回報
3. 適當的學習率
4. 添加 entropy bonus 增加探索

訓練技巧:
1. 梯度裁剪
2. 多環境並行
3. 適當的網路大小
4. 監控訓練曲線
```

### 下一步

前往 `deployment/model_compression.ipynb` 學習模型壓縮！

---
## 🏋️ 練習

### 練習 1: 比較不同超參數對 REINFORCE 的影響

測試不同學習率和折扣因子對訓練效果的影響。

In [ ]:
# ========== 練習 1: 超參數實驗 ==========

def train_reinforce_short(env, lr, gamma, n_episodes=200):
    """
    訓練 REINFORCE 並返回學習曲線
    """
    agent = REINFORCE(state_dim, action_dim, lr=lr, gamma=gamma)
    episode_rewards = []
    
    for episode in range(n_episodes):
        state = env.reset() if not HAS_GYM else env.reset()[0]
        episode_reward = 0
        done = False
        
        while not done:
            action = agent.select_action(state)
            result = env.step(action)
            if HAS_GYM:
                next_state, reward, terminated, truncated, _ = result
                done = terminated or truncated
            else:
                next_state, reward, done, _ = result
            agent.store_reward(reward)
            state = next_state
            episode_reward += reward
        
        agent.update()
        episode_rewards.append(episode_reward)
    
    # 返回滑動平均
    window = 20
    avg_rewards = [np.mean(episode_rewards[max(0,i-window):i+1]) for i in range(len(episode_rewards))]
    return avg_rewards

def experiment_hyperparameters():
    """
    比較不同超參數組合
    """
    configs = [
        {'lr': 1e-2, 'gamma': 0.99, 'label': 'lr=1e-2, γ=0.99'},
        {'lr': 1e-3, 'gamma': 0.99, 'label': 'lr=1e-3, γ=0.99'},
        {'lr': 1e-4, 'gamma': 0.99, 'label': 'lr=1e-4, γ=0.99'},
        {'lr': 1e-3, 'gamma': 0.9, 'label': 'lr=1e-3, γ=0.9'},
        {'lr': 1e-3, 'gamma': 0.95, 'label': 'lr=1e-3, γ=0.95'},
    ]
    
    results = {}
    
    print("練習 1: REINFORCE 超參數實驗")
    print("=" * 50)
    
    for config in configs:
        print(f"訓練中: {config['label']}...")
        rewards = train_reinforce_short(env, config['lr'], config['gamma'], n_episodes=200)
        results[config['label']] = rewards
        print(f"  最終平均獎勵: {rewards[-1]:.1f}")
    
    return results

# 執行實驗
results = experiment_hyperparameters()

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 學習曲線比較
colors = plt.cm.tab10(np.linspace(0, 1, len(results)))
for (label, rewards), color in zip(results.items(), colors):
    axes[0].plot(rewards, label=label, color=color, alpha=0.8)

axes[0].axhline(y=200, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Average Reward (window=20)')
axes[0].set_title('Learning Curves with Different Hyperparameters')
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

# 最終表現比較
labels = list(results.keys())
final_rewards = [results[l][-1] for l in labels]
bars = axes[1].barh(range(len(labels)), final_rewards, color=colors)
axes[1].set_yticks(range(len(labels)))
axes[1].set_yticklabels([l.split(',')[0] for l in labels], fontsize=9)
axes[1].set_xlabel('Final Average Reward')
axes[1].set_title('Final Performance Comparison')

for i, (bar, reward) in enumerate(zip(bars, final_rewards)):
    axes[1].text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
                 f'{reward:.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n結論：")
print("- 學習率太高可能導致不穩定，太低收斂慢")
print("- 折扣因子 γ 影響對長期獎勵的重視程度")
print("- CartPole 通常 lr=1e-3, γ=0.99 效果較好")